In [10]:
import numpy as np
from shapely.geometry import Point, Polygon
import scipy
import time
import scipy.ndimage
import matplotlib.pyplot as plt

In [12]:
def where_is_point_1(point, fractal,x,y):
    '''
    Siden vi skal se på naboa seinere kanskje greit å gjøre det punkt for punkt??
    Don't know bare prøva nokke hehe, confusing oppgave
    '''
    
    #0 = outside, 1 = inside, 2 = on boundary
    inside_x = 0
    inside_y = 0
    
    #The point we are testing for
    x,y = point

    #The y-points on the fractal corresponding the x-value of the point
    y_values = []

    ink_y = 0
    for i in range(len(fractal)):
        if fractal[i,0] == x:
            y_values.append(fractal[i,1])
            ink_y += 1

    #print(y_values)

    if len(y_values) > 1:
        if y < np.max(y_values) and y > np.min(y_values) and (y not in y_values):
            inside_y = 1
        elif (y in y_values):
            inside_y = 2

    #The x-points on the fractal corresponding the y-value of the point
    x_values = []

    ink_x = 0
    for i in range(len(fractal)):
        if fractal[i,1] == y:
            x_values.append(fractal[i,0])
            ink_x += 1

    if len(x_values) > 1:
        if x < np.max(x_values) and x > np.min(x_values) and (x not in x_values):
            inside_x = 1
        elif (x in x_values):
            inside_x = 2

    if (inside_x == 1 and inside_y == 1) or (inside_x == 1 and inside_y == 2) or (inside_x == 2 and inside_y == 1):
        return 1
    elif inside_x == 2 and inside_y == 2:
        return 2
    else:
        return 0

In [13]:
def num_of_point_inside_fractal(grid, fractal, parameter,x,y,number=1):
    '''
    parameter: 0 for outside, 1 for inside, 2 for boundary
    Total number of points
    '''
    start_time = time.time()
    num_points = 0
   
    if number == 1:
        for i in range(len(grid)):
            if where_is_point_2(grid[i], fractal,x,y) == parameter:
                num_points += 1
    if number == 2:
        for i in range(len(grid)):
            if where_is_point_3(grid[i], fractal,x,y) == parameter:
                num_points += 1
                
    end_time = time.time()
    runtime = end_time-start_time
    return num_points, runtime

In [1]:
def where_is_point_2(point, fractal, x, y):
    '''
    Method 1
    where: 0 is outside, 1 is inside, 2 is boundary
    '''
    where = 0

    polygon = Polygon(fractal)

    test_point = Point(point)

    if polygon.contains(test_point):
        where = 1
    elif test_point.within(polygon.boundary):
        where = 2

    return where

In [ ]:
def where_is_point_3(point,fractal_n,x,y):
    ''' 
    Find point inside, border and outside
    '''

    index_x, index_y = np.where(x == point[0]), np.where(y == point[1])


    boolean_grid_boundary = make_boolean_grid(x,y,fractal_n)
    boolean_grid_inside = scipy.ndimage.binary_fill_holes(boolean_grid_boundary).astype(int) - boolean_grid_boundary

    if boolean_grid_boundary[index_x, index_y ] == 1:
        return 2
    elif boolean_grid_inside[index_x, index_y ] == 1:
        return 1
    else:
        return 0

In [9]:
def plot_all_points(fractal_a, method):
    '''
    Parameters:
        fractal = The generated fractal.
        method = One of the two methods above.

    Function that loops throud all points in the grid and checks if they are inside, outside or on the border of the fractal. The result is presented in a scatter plot
    where the yellow dots represent outside, the magneta ones are inside, and finally the green ones represent points on the border. In order to compare the two
    methods the time it takes for the code to run is also calulated.
    '''
    
    start_time = time.time()
    
    x_a, y_a = make_lattice(fractal_a)
    
    outside = []
    inside = []
    border = []

    value = 0

    for i in range(len(x_a)):
        for j in range(len(y_a)):

            point = [x_a[i], y_a[j]]
            
            value = method(point, fractal_a, x_a, y_a)
            
            if value == 2:
                border.append(point)
            elif value == 1:
                inside.append(point)
            else:
                outside.append(point)
    
    
    plt.figure(figsize=(7,7))

    for k in range(len(outside)):
        plt.scatter(outside[k][0],outside[k][1], s = 10, c = 'y')
    
    for l in range(len(inside)):
        plt.scatter(inside[l][0],inside[l][1], s = 10, c = 'r')
    
    for m in range(len(border)):
        plt.scatter(border[m][0],border[m][1], s = 10, c = 'b')

    end_time = time.time()

    plt.title(f"Position of all points. Runtime = {round(end_time - start_time, 2)}s", fontsize = 20)

    plt.plot(fractal_a[:,0], fractal_a[:,1])
    plt.savefig('/Users/simon/Downloads/numfys/A3/images/task2/L4fractalLatticeColored.png')

    
    plt.show()